<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/Kindle%20Clippings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kindle Clippings

xxx

## First run:

Go to "Setup and Configuration" and check "inital_setup_mode". Then run all cells. Afterwards uncheck "Setup and Configuration".

# Setup and Configuration

In [2]:
# @title Settings

# @markdown Default model
default_model = 'GPT-3.5' # @param ["GPT-3.5", "GPT-4"]

# @markdown Initial Setup Mode for pip install, fetch credentials etc.
initial_setup_mode = False # @param {type:"boolean"}

# @markdown Debug Mode for extensive logging.
debug_mode = True # @param {type:"boolean"}

# @markdown iOS Mode to develop helper functions, no Gradio.
# @markdown Useful for development on iOS, eg. with Carnets App
ios_mode = False # @param {type:"boolean"}

In [ ]:
folders = {
    'audio': 'audio',
    'audio-chunks': 'audio/chunks',
    'transcript':'audio-transcript',
    'transcript-chunks': 'audio-transcript/chunks',
    'text-input': 'text-input',
    'text-input-backup': 'text-input-backup'
}

In [ ]:
if ios_mode == False:
  print('Mac')
else:
  print('iOS')

Mac


In [ ]:
if initial_setup_mode == True:
  !pip install openai==0.27.7 yt-dlp==2023.7.6 librosa==0.10.0.post2 pickle-mixin==1.0.2 langchain==0.0.225 PyPDF2==3.0.1 PyMuPDF==1.22.5 -q
else:
  print('No initial setup.')


No initial setup.


In [ ]:
if (initial_setup_mode == True) and (ios_mode == False) :
  !pip install gradio -q
else:
  print('No initial setup / iOS.')

No initial setup / iOS.


In [ ]:
if (initial_setup_mode == True) and (ios_mode == False) :
  %load_ext gradio
else:
  print('No initial setup / iOS.')

No initial setup / iOS.


In [ ]:
if initial_setup_mode == True:
  !wget -q bit.ly/aknip-colab-setup
  %run aknip-colab-setup
else:
  print('No initial setup.')

No initial setup.


In [ ]:
creds = json.loads(os.getenv('CREDS'))
key = creds['OpenAI']['v2']['credential']
# print(key)

# Helper Functions

- **create_file_directory**: Creates a new directory - if it not exists yet. The always_delete flag forces a deletion even if it exists.

In [11]:
import shutil
import os

def create_file_directory(directory, always_delete=False):
  # Creates a new directory - if it not exists yet. The always_delete flag forces a deletion even if it exists.
  # Examples:
  # - create_file_directory('texts', False) => creates a new directory only if it not exists yet
  # - create_file_directory('texts', True) => always deletes existing directory and creates a new one
  if os.path.exists(directory):
    if always_delete:
      # delete the diectory recursively
      shutil.rmtree(directory)
  # create directory
  if not os.path.exists(directory):
    os.mkdir(directory)



# The App

In [35]:
if ios_mode == False:
  import re
  import hashlib
  from dateutil.parser import parse
  import os
  from datetime import datetime, timedelta, timezone
  import getpass
  import sys

  infile = 'My Clippings.txt'
  infile_fixed = 'My Clippings-fixed.txt'
  outpath = 'clippings/'

  f= open(infile,'r')
  if f.mode == 'r':
        contents =f.read()
  f.close()
  contents2 = contents.replace(chr(65279), "")
  f= open(infile_fixed,'w+')
  f.write(contents2)
  f.close()

  create_file_directory('clippings', True)

  def getvalidfilename(filename):
      import unicodedata
      clean = unicodedata.normalize('NFKD', filename)
      return re.sub('[^\w\s()\'.?!:-]', '', clean)


  note_sep = '=========='

  commentstr = '.. '  # RST (reStructuredText) comment

  #regex_title = re.compile('^(.*)\((.*)\)$')
  #regex_info = re.compile(r'^- (\S+) (.*)[\s|]+Added on\s+(.+)$')
  #regex_loc = re.compile('Loc\. ([\d\-]+)')
  #regex_page = re.compile('Page ([\d\-]+)')
  #regex_date = re.compile('Added on\s+(.+)$')

  regex_title = re.compile('^(.*)\((.*)\)$')
  regex_info = re.compile(r'^- (\S+) (.*)[\s|]+Hinzugefügt am\s+(.+)$')
  regex_loc = re.compile('bei Position ([\d\-]+)')
  regex_page = re.compile('Seite ([\d\-]+)')
  regex_date = re.compile('Hinzugefügt am\s+(.+)$')

  regex_hashline = re.compile('^\.\.\s*([a-fA-F0-9]+)' + '\s*')


  pub_title = {}
  pub_author = {}
  pub_notes = {}
  pub_hashes = {}

  notes = {}
  locations = {}
  types = {}
  dates = {}

  existing_hashes = {}

  print('Scanning output dir', outpath)
  for directory, subdirlist, filelist in os.walk(outpath):
      for fname in filelist:
          ext = fname[-4:]
          if ext == '.rst' or ext == '.RST':
              print('Found RST file', fname, 'in directory', directory)
              # open file, find commend lines, store hashes
              rst = open(directory + '/' + fname, 'r')
              line = rst.readline()
              lines = 0
              hashes = 0
              while line:
                  lines += 1
                  findhash_result = regex_hashline.findall(line)
                  if len(findhash_result):
                      foundhash = findhash_result[0]
                      existing_hashes[foundhash] = fname
                      hashes += 1
                  line = rst.readline()
              rst.close()
              print(hashes, 'hashes found in', lines, 'scanned lines')
          else:
              print('File', fname, 'does not seem to be RST, skipping', ext)

  print('Found', len(existing_hashes), 'existing note hashes')
  print('Processing clippings file', infile)

  mc = open(infile_fixed, 'r')

  mc.read(0)  # Was initially: mc.read(1) Skip first character - not necessary? Fixed with 0

  line = mc.readline().strip()

  while line:

      key = line.strip()
      result_title = regex_title.findall(key)    # Extract title and author
      line = mc.readline().strip()                # Read information line
      note_type, location, date = regex_info.findall(line)[0]    # Extract note type, location and date
      result_loc = regex_loc.findall(location)
      result_page = regex_page.findall(location)
      if len(result_title):
          title, author = result_title[0]
      else:
          title = key
          author = 'Unknown'

      if len(result_loc):
          note_loc = result_loc[0]
      else:
          note_loc = ''

      if len(result_page):
          note_page = result_page[0]
      else:
          note_page = ''

      note_text = ''
      line = mc.readline()                # Skip empty line
      line = mc.readline().strip()

      while line != note_sep:
          note_text += line + '\n'
          line = mc.readline().strip()

      note_hash = hashlib.sha256(note_text.strip().encode('utf8')).hexdigest()[:8]

      if key not in pub_notes:
          pub_notes[key] = []
          pub_hashes[key] = []

      pub_title[key] = title.strip()
      pub_author[key] = author.strip()
      pub_notes[key].append(note_text.strip())
      pub_hashes[key].append(note_hash)

      locstr = ''
      if note_loc:
          locstr = 'loc.' + note_loc
      if note_page:
          if note_loc:
              locstr += ', '
          locstr += 'p.' + note_page

      try:
          datestr = str(parse(date))
      except:
          datestr = date

      notes[note_hash] = note_text.strip()
      locations[note_hash] = locstr
      types[note_hash] = note_type
      dates[note_hash] = datestr

      line = mc.readline().strip()

  mc.close()

  for key in pub_title.keys():
      nr_notes = len(pub_notes[key])
      author = pub_author[key]
      title = pub_title[key]
      short_title = title.split('|')[0]
      short_title = short_title.split(' - ')[0]
      short_title = short_title.split('. ')[0]
      if len(short_title) > 128:
          short_title = short_title[:127]

      fname = author + ' - ' + short_title.strip() + '.rst'
      short = 0
      #if (nr_notes > 2):
      #    fname = author + ' - ' + short_title.strip() + '.rst'
      #    short = 0
      #else:
      #    fname = 'short_notes.rst'
      #    short = 1

      new_hashes = 0
      for note_hash in pub_hashes[key]:
          if note_hash not in existing_hashes:
              new_hashes += 1

      if new_hashes > 0:
          print(new_hashes, 'new notes found for', title)
      else:
          continue            # Skip to next title if there are no new hashes

      outfile = os.path.join(outpath, getvalidfilename(fname))

      newfile = os.path.isfile(outfile)

      out = open(outfile, 'a')

      if short:
          # Short note, output a small header and append to short note file
          if author != 'Unknown':
              titlestr = author + ' - ' + title
          else:
              titlestr = title
          out.write(titlestr + '\n')
          out.write(('-' * len(titlestr)) + '\n\n')
      elif not newfile:
          # Many notes, output with header and metadata in a separate file
          titlestr = 'Highlights from ' + title
          out.write(titlestr + '\n')
          out.write(('=' * len(titlestr)) + '\n\n')
          if author != 'Unknown':
              out.write('Authors:: [[' + author + ']]' + '\n')
          out.write('Recommended By:: \nTags:: [[Books]]\n\n# ' + title + '\n\n### Highlights\n')

      last_date = datetime.now()

      for note_hash in pub_hashes[key]:
          note = notes[note_hash]
          note_type = types[note_hash]
          note_date = dates[note_hash]
          note_loc = locations[note_hash]
          if note_hash in existing_hashes:
              print('Note', note_hash, 'is already in', existing_hashes[note_hash])
          else:
              print('Adding new note to', outfile + ':', note_hash, note_type, note_loc, note_date)

              comment = str(commentstr + note_hash + ' ; ' + note_type + ' ; ' + note_loc + ' ; ' + note_date)

              if short:
                  comment += ' ; ' + author + ' ; ' + title

              # this adds metadata before each note.
              # out.write(comment + '\n\n')
              out.write('- ' + note + '\n')
          try:
              last_date = parse(note_date)
          except:
              pass

      out.close()

      # Update file modification time to time of last note

      if last_date.tzinfo is None or last_date.tzinfo.utcoffset(last_date) is None:
          epoch = datetime(1970, 1, 1)
      else:
          epoch = datetime(1970, 1, 1, tzinfo=timezone.utc)
      note_timestamp = (last_date - epoch) / timedelta(seconds=1)
      os.utime(outfile, (note_timestamp, note_timestamp))

      shutil.make_archive('archive', 'zip', outpath)

else:
  print('iOS Mode - Nothing to do.')

Scanning output dir clippings/
Found 0 existing note hashes
Processing clippings file My Clippings.txt
4 new notes found for ChatGPT for Better Business Communication: How to Use ChatGPT to Increase Productivity and Communicate More Effectively at Work (ChatGPT prompts, tips, and examples that help you in the workplace)
Adding new note to clippings/Osman Hassan - ChatGPT for Better Business Communication: How to Use ChatGPT to Increase Productivity and Communicate More Effectively at Work.rst: 207abae3 Deine loc.198-206, p.13 Mittwoch, 2. August 2023 08:42:25
Adding new note to clippings/Osman Hassan - ChatGPT for Better Business Communication: How to Use ChatGPT to Increase Productivity and Communicate More Effectively at Work.rst: 6a6bd197 Deine loc.264-267, p.18 Mittwoch, 2. August 2023 08:42:41
Adding new note to clippings/Osman Hassan - ChatGPT for Better Business Communication: How to Use ChatGPT to Increase Productivity and Communicate More Effectively at Work.rst: 36b510f9 Dein

In [ ]:
f= open('My Clippings.txt','r')
if f.mode == 'r':
      contents =f.read()
f.close()

print(contents)
print(len(contents))
print(contents[2])
print(ord(contents[2]))
print(contents[1])
print(ord(contents[1]))
print(chr(65279))

contents2 = contents.replace(chr(65279), "")
print(contents2)
print(len(contents2))

f= open('My Clippings2.txt','w+')
f.write(contents2)
f.close()
